In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.5`
import $ivy.`org.apache.spark::spark-streaming:2.4.5`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._

val spark = {
  NotebookSparkSession.builder()
    .config("spark.sql.shuffle.partitions", 1)
    .config("parquet.enable.summary-metadata", "false")
    .config("spark.sql.streaming.checkpointLocation", "checkpoints")
    .master("local[*]")
    .getOrCreate()
}

import scala.reflect.io.Directory
import java.io.File

new Directory(new File("tables")).deleteRecursively()

val dataSourceFile = "./sample_data.json"
val correctDataSourceFile = "./correct_data.json"

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

import $ivy.$                                  

import $ivy.$                                        


import org.apache.log4j.{Level, Logger}

import org.apache.spark.sql._


spark: SparkSession = org.apache.spark.sql.SparkSession@74579c87
import scala.reflect.io.Directory

import java.io.File


res0_8: Boolean = false
dataSourceFile: String = "./sample_data.json"
correctDataSourceFile: String = "./correct_data.json"

# Lambda

## Streaming layer

In [2]:
import org.apache.spark.sql.execution.streaming.MemoryStream
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._

import spark.implicits._

implicit val sqlContext = spark.sqlContext

val sampleDataStream = MemoryStream[String]

val schema = new StructType()
    .add(StructField("timestamp", LongType))
    .add(StructField("page", StringType))
    .add(StructField("user_id", LongType))

def streamingLayer(dataStream: MemoryStream[String], outputDir: String) = {

    def loadStream = dataStream
        .toDF
        .select(from_json($"value", schema) as "value")
        .select("value.*")
        .withColumn("timestamp", to_timestamp(from_unixtime('timestamp)))
        .withWatermark("timestamp", "1 second")

    def streamingLayer = loadStream
        .withColumn("event_date", to_date('timestamp))
        .groupBy(window('timestamp, "1 minute"), 'event_date, 'page)
        .count

    streamingLayer
        .writeStream
        .outputMode("append")
        .format("parquet")
        .partitionBy("event_date")
        .option("path", outputDir)
        .start()
    
}

val streamingQuery = streamingLayer(sampleDataStream, "tables/streaming_table")

import org.apache.spark.sql.execution.streaming.MemoryStream

import org.apache.spark.sql.functions._

import org.apache.spark.sql.types._


import spark.implicits._


sqlContext: SQLContext = org.apache.spark.sql.SQLContext@3bde0c63
sampleDataStream: MemoryStream[String] = MemoryStream(
  0,
  org.apache.spark.sql.SQLContext@3bde0c63
)
schema: StructType = StructType(
  StructField("timestamp", LongType, true, {}),
  StructField("page", StringType, true, {}),
  StructField("user_id", LongType, true, {})
)
defined function streamingLayer
streamingQuery: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@5ae1c99a

In [3]:
import scala.io.Source

def loadDataToStream(filename: String, stream: MemoryStream[String]) = {
    Source
        .fromFile(filename)
        .getLines
        .foreach(s => stream.addData(s))
}

loadDataToStream(dataSourceFile, sampleDataStream)
streamingQuery.processAllAvailable()

start at cmd1.sc:35

7 / 7

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

...

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

1 / 1

import scala.io.Source


defined function loadDataToStream

In [4]:
spark.read.parquet("tables/streaming_table").show(false)

parquet at cmd3.sc:1

1 / 1

show at cmd3.sc:1

1 / 1

show at cmd3.sc:1

1 / 1

+------------------------------------------+-------+-----+----------+
|window                                    |page   |count|event_date|
+------------------------------------------+-------+-----+----------+
|[2020-05-03 15:47:00, 2020-05-03 15:48:00]|catalog|3    |2020-05-03|
|[2020-05-03 15:47:00, 2020-05-03 15:48:00]|payment|1    |2020-05-03|
|[2020-05-03 15:47:00, 2020-05-03 15:48:00]|order  |1    |2020-05-03|
|[2020-05-03 15:47:00, 2020-05-03 15:48:00]|product|1    |2020-05-03|
|[2020-05-02 15:47:00, 2020-05-02 15:48:00]|catalog|1    |2020-05-02|
+------------------------------------------+-------+-----+----------+



## Batch layer

In [5]:
import $ivy.`joda-time:joda-time:2.10.5`

import org.joda.time.LocalDate
import org.apache.spark.sql.functions._


def processDay(filename: String, executionDateString: String) = {
    
    val executionDate = LocalDate.parse(executionDateString)
    
    val startUnixTime = executionDate.toDateTimeAtStartOfDay.getMillis / 1000
    val endUnixTime = executionDate.plusDays(1).toDateTimeAtStartOfDay.getMillis / 1000
    
    import spark.implicits._
    
    def loadBatch = spark
        .read 
        .json(filename)
        .where('timestamp between (startUnixTime, endUnixTime))
        
    loadBatch
        .withColumn("event_date", to_date(from_unixtime('timestamp)))
        .groupBy('event_date, 'page)
        .count
        .write
        .mode("overwrite")
        .parquet(s"tables/batch_table/event_date=$executionDateString")
}

processDay(dataSourceFile, "2020-05-02")
processDay(dataSourceFile, "2020-05-02")
processDay(dataSourceFile, "2020-05-02")

json at cmd4.sc:18

1 / 1

parquet at cmd4.sc:27

1 / 1

parquet at cmd4.sc:27

...

parquet at cmd4.sc:27

1 / 1

parquet at cmd4.sc:27

1 / 1

import $ivy.$                           


import org.joda.time.LocalDate

import org.apache.spark.sql.functions._



defined function processDay

## Serving layer

In [6]:
spark.read.parquet("tables/streaming_table").createOrReplaceTempView("streaming_table")
spark.read.parquet("tables/batch_table").createOrReplaceTempView("batch_table")

spark.sql("""
    SELECT event_date, page, count
    FROM batch_table
    UNION ALL (
        SELECT event_date, page, SUM(count) as count
        FROM streaming_table
        WHERE event_date > (SELECT MAX(event_date) FROM batch_table)
        GROUP BY event_date, page
    )
    ORDER BY event_date, page
""").createOrReplaceTempView("lambda_serving_layer")

spark.sql("SELECT * FROM lambda_serving_layer").show

parquet at cmd5.sc:1

1 / 1

parquet at cmd5.sc:2

1 / 1

run at ThreadPoolExecutor.java:1149

...

show at cmd5.sc:16

2 / 2

show at cmd5.sc:16

2 / 2

+----------+-------+-----+
|event_date|   page|count|
+----------+-------+-----+
|2020-05-02|catalog|    1|
|2020-05-03|catalog|    3|
|2020-05-03|  order|    1|
|2020-05-03|payment|    1|
|2020-05-03|product|    1|
+----------+-------+-----+



## Reprocessing

In [7]:
import scala.reflect.io.Directory
import java.io.File

val dateToReprocess = "2020-05-02"
val directory = new Directory(new File(s"tables/batch_table/event_date=$dateToReprocess"))
directory.deleteRecursively()

processDay(correctDataSourceFile, "2020-05-02")

spark.sql("REFRESH TABLE batch_table")
spark.sql("SELECT * FROM lambda_serving_layer").show

json at cmd4.sc:18

1 / 1

parquet at cmd4.sc:27

1 / 1

parquet at cmd4.sc:27

...

2 / 2

show at cmd6.sc:11

2 / 2

+----------+-------+-----+
|event_date|   page|count|
+----------+-------+-----+
|2020-05-02|catalog|    2|
|2020-05-03|catalog|    3|
|2020-05-03|  order|    1|
|2020-05-03|payment|    1|
|2020-05-03|product|    1|
+----------+-------+-----+



import scala.reflect.io.Directory

import java.io.File


dateToReprocess: String = "2020-05-02"
directory: Directory = tables/batch_table/event_date=2020-05-02
res6_4: Boolean = true
res6_6: DataFrame = []

# Kappa

## Streaming layer

In [8]:
val sampleDataStream2 = MemoryStream[String]

val streamingQuery2 = streamingLayer(sampleDataStream2, "tables/kappa_table")
loadDataToStream(dataSourceFile, sampleDataStream2)
streamingQuery2.processAllAvailable()

start at cmd1.sc:35

8 / 8

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

1 / 1

sampleDataStream2: MemoryStream[String] = MemoryStream(
  1,
  org.apache.spark.sql.SQLContext@3bde0c63
)
streamingQuery2: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@2a33c6f

In [9]:
spark.read.parquet("tables/kappa_table").createOrReplaceTempView("kappa_table")
spark.sql("""
    SELECT event_date, page, SUM(count) as count
    FROM kappa_table
    GROUP BY event_date, page
    ORDER BY event_date, page
""").createOrReplaceTempView("kappa_serving_layer")

spark.sql("SELECT * FROM kappa_serving_layer").show

parquet at cmd8.sc:1

1 / 1

show at cmd8.sc:9

2 / 2

show at cmd8.sc:9

1 / 1

+----------+-------+-----+
|event_date|   page|count|
+----------+-------+-----+
|2020-05-02|catalog|    1|
|2020-05-03|catalog|    3|
|2020-05-03|  order|    1|
|2020-05-03|payment|    1|
|2020-05-03|product|    1|
+----------+-------+-----+



## Reprocessing

In [10]:
import scala.reflect.io.Directory
import java.io.File

// Delete old Kappa
streamingQuery2.stop()
new Directory(new File("tables/kappa_table")).deleteRecursively()

// Re-create Streaming layer
val sampleDataStream3 = MemoryStream[String]
val streamingQuery3 = streamingLayer(sampleDataStream3, "tables/kappa_table")
loadDataToStream(correctDataSourceFile, sampleDataStream3)
streamingQuery3.processAllAvailable()

start at cmd1.sc:35

9 / 9

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

1 / 1

start at cmd1.sc:35

1 / 1

import scala.reflect.io.Directory

import java.io.File

// Delete old Kappa

res9_3: Boolean = true
sampleDataStream3: MemoryStream[String] = MemoryStream(
  2,
  org.apache.spark.sql.SQLContext@3bde0c63
)
streamingQuery3: streaming.StreamingQuery = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@6de67bb8

In [11]:
// Re-create Serving layer
spark.read.parquet("tables/kappa_table").createOrReplaceTempView("kappa_table")
spark.sql("""
    SELECT event_date, page, SUM(count) as count
    FROM kappa_table
    GROUP BY event_date, page
    ORDER BY event_date, page
""").createOrReplaceTempView("kappa_serving_layer")

spark.sql("SELECT * FROM kappa_serving_layer").show

parquet at cmd10.sc:1

1 / 1

show at cmd10.sc:9

2 / 2

show at cmd10.sc:9

1 / 1

+----------+-------+-----+
|event_date|   page|count|
+----------+-------+-----+
|2020-05-02|catalog|    2|
|2020-05-03|catalog|    3|
|2020-05-03|  order|    1|
|2020-05-03|payment|    1|
|2020-05-03|product|    1|
+----------+-------+-----+

